In [12]:
using WAV, LossFunctions

LoadError: [91mArgumentError: Module LossFunctions not found in current path.
Run `Pkg.add("LossFunctions")` to install the LossFunctions package.[39m

In [2]:
music_sample_size = 479232
sample_frequency = 16000
batch_size = 5

function load_data()
    X = Array{Array{Float32,1},1}()
    for musician in ["satriani"]
        for sample in readdir("/mnt/data/other/music_samples/processed/$(musician)")
            values_raw, fq = wavread("/mnt/data/other/music_samples/processed/$(musician)/$(sample)")
            if length(values_raw[:,1]) >= music_sample_size
                push!(X, values_raw[:,1][1:music_sample_size])
            end
        end
    end
    Array{Float32,2}(hcat(X...))
end

X = load_data()
println("Loaded $(length(X[1,:])) samples !")

Loaded 95 samples !


In [3]:
function create_D()
    samples = music_sample_size
    Weights = [
        xavier(Float32,Int64(samples*2/(3*512)),samples),
        xavier(Float32,Int64(samples/1024),Int64(samples*2/(3*512))), 
        xavier(Float32,Int64(samples/2048),Int64(samples/1024)), 
        xavier(Float32,2,Int64(samples/2048))
    ]
    Biases = [
        zeros(Float32,samples*2/(3*512),1),
        zeros(Float32,samples/1024,1),
        zeros(Float32,samples/2048,1),
        zeros(Float32,2,1)
    ]
    return Weights, Biases
end

function create_G()
    granularity = 64
    oscilators = 80
    samples = music_sample_size
    Weights = [
        xavier(Float32,Int64(oscilators * granularity), Int64(oscilators * granularity)),
        xavier(Float32,Int64(oscilators * granularity / 2), Int64(oscilators * granularity)), 
        xavier(Float32,Int64(oscilators * granularity / 4), Int64(oscilators * granularity / 2)), 
        xavier(Float32,Int64(oscilators * granularity), Int64(oscilators * granularity / 4))
    ]
    Biases = [
        xavier(Float32,Int64(oscilators * granularity), 1),
        xavier(Float32,Int64(oscilators * granularity / 2), 1),
        xavier(Float32,Int64(oscilators * granularity / 4), 1),
        xavier(Float32,Int64(oscilators * granularity), 1)
    ]
    return Weights, Biases, granularity, granularity
end

create_generator (generic function with 1 method)

In [13]:
lReLu(x;α=Float32(0.3)) = max(0,x) + α*min(0,x)
dlReLu(x;α=Float32(0.3)) = x < 0 ? α : 1

softmax(x) = exp(x)./sum(exp(x))
dsoftmax(x) =x.*(1.0-x)

dsoftmax (generic function with 1 method)

In [ ]:
global const ε = Float32(1e-7)

function forward_prop_d(W,B,X)
    for i=1:length(W) - 1
        X = lReLu.(W[i]*X .+ B[i])
    end
    softmax.(W[end]*X .+ B[end])
end

Jd(W,B,X,X_fake) = -mean(log.(forward_prop_d(W,B,X) + ε) + log.(1 - forward_prop_d(W,B,X_fake) + ε))
Jg(Wg,Bg,Wd,Bd,X) = -mean(log.(forward_prop_d(Wd,Bd,forward_prop_g(Wg,Bg,X)) + ε))

function backward_prop_d(W,B,X,X_fake)
    loss = Jd(W,B,X,X_fake)
    
    Errors = Vector()
    push!(Errors, loss)
    
    push!(Errors, (W[end]*Errors[end]).*dsoftmax().*)
    
    E = softmax.(W[end]*X .+ B[end])
    for i=reverse(1:length(W) - 1)
        push!(Errors, W[end] - loss)
        X = lReLu.(W[i]*X .+ B[i])
    end
    
end

function forward_prop_g(W,B,X)
    for i=1:length(W) - 1
        X = lReLu.(W[i]*X .+ B[i])
    end
    softmax.(W[end]*X .+ B[end])
    
    # Create the actual music using the values
    for i=1:nr_smaples
        vals = [] 
        for sample_nr=0:oscilators:oscilators*(granularity - 1)
            push!(vals, Z[(sample_nr + 1):(sample_nr + oscilators), i])        
        end

        for si=1:music_sample_size
            oi = 1
            for v in vals[Int64(round(1 + si/music_sample_size * (granularity -1) ))]
                samples[si, i] += sin(si*(oi^(2.3))/16000)*v
                oi += 1
            end
            samples[si, i] = samples[si, i]/oscilators
        end
    end
    return samples
end